"""
Steps we will follow to create  our mcq generator
<!--  -->
import libraries
create response_json means the pattern which we want for our mcq generator
Template : quiz template
mention chain
write second template for review
chain
"""

In [2]:
import os
import json
import pandas as pd
import traceback


In [15]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [4]:
from langchain.chat_models import ChatOpenAI

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
KEY=os.getenv("OPENAI_API_KEY") # create a file with name .env and paste the key in .env 

In [8]:
print(KEY)

None


In [19]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.1) # high the value of temperature more will be creatvity of the  response

In [36]:
RESPONSE_JSON= {
    "questions": {
        "1": {
            "question": "multiple choice question",
            "options": {
                "a": "choice here",
                "b": "choice here",
                "c": "choice here",
                "d": "choice here"
            },
            "correct_answer": "correct answer"
        },
        "2": {
            "question": "multiple choice question",
            "options": {
                "a": "choice here",
                "b": "choice here",
                "c": "choice here",
                "d": "choice here"
            },
            "correct_answer": "correct answer"
        },
        "3": {
            "question": "multiple choice question",
            "options": {
                "a": "choice here",
                "b": "choice here",
                "c": "choice here",
                "d": "choice here"
            },
            "correct_answer": "correct answer"
        }
    }
}


In [12]:
#quiz temp
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Based on the provided text, your task is to create a quiz consisting of {number} multiple-choice questions for {subject} students in a {tone} tone.
Ensure that the questions are unique, relevant, and cover various aspects of the text. Each question should have four options labeled as "Option A", "Option B", "Option C", and "Option D", with one correct answer. 

Follow the format given in the RESPONSE_JSON example below and adhere to it while creating the MCQs. 

### RESPONSE_JSON
{response_json}

Make sure your questions:
- Are clear and concise.
- Have four distinct options.
- Indicate the correct answer using the appropriate key ("a", "b", "c", or "d").

Ensure that you generate exactly {number} MCQs.
"""


In [16]:
# Initialize the PromptTemplate with the TEMPLATE string
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [20]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [21]:
TEMPLATE2 = """
You are an expert English grammarian and educational writer. Given a multiple-choice quiz designed for {subject} students, your task is to:

1. **Analyze the Complexity**: Assess the cognitive and analytical demands of the questions and provide a brief analysis of the quiz’s complexity in up to 50 words.

2. **Evaluate and Update**: Review the quiz to ensure that it aligns with the students’ cognitive and analytical abilities. If the quiz is not appropriate, update the questions as needed and adjust the tone to better fit the students' needs.

3. **Check the Quiz**: Provide feedback on the quiz from the perspective of an expert English writer.

Quiz_MCQs:
{quiz}

Complexity Analysis:
"""


In [22]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [25]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [29]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [30]:
file_path=r"E:\VSCode_Projects\experiment\data.txt"

In [31]:
file_path

'E:\\VSCode_Projects\\experiment\\data.txt'

In [33]:
with open(file_path, 'r') as file:
    TEXT = file.read()


In [34]:
print(TEXT)

1. Understanding Generative AI: Definition and Technology
Generative AI refers to a subset of artificial intelligence designed to produce new content, data, or solutions that mimic human creativity. Unlike traditional AI systems that rely on fixed algorithms and predefined rules, generative AI leverages complex machine learning techniques to create novel outputs. At the heart of this technology are neural networks, specifically deep learning models such as Generative Adversarial Networks (GANs) and Transformer architectures. GANs consist of two networks: a generator that creates new data instances and a discriminator that evaluates them, with the goal of producing data that is indistinguishable from real-world examples. On the other hand, Transformer-based models like GPT-3 and GPT-4 use vast amounts of text data to generate coherent and contextually relevant text by predicting the next word in a sequence based on prior words. These models are trained on diverse datasets, including tex

In [37]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"questions": {"1": {"question": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct_answer": "correct answer"}, "2": {"question": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct_answer": "correct answer"}, "3": {"question": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct_answer": "correct answer"}}}'

In [38]:
NUMBER=5 
SUBJECT="Artificial Intelligence"
TONE="Humble anf informative"

In [40]:
# #How to setup Token Usage Tracking in LangChain
# with get_openai_callback() as cb:
#     response=generate_evaluate_chain(
#         {
#             "text": TEXT,
#             "number": NUMBER,
#             "subject":SUBJECT,
#             "tone": TONE,
#             "response_json": json.dumps(RESPONSE_JSON)
#         }
#         )

In [41]:
# we can take reference of langchain documentation for more info

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )
    

In [43]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2407

Prompt Tokens:1608

Completion Tokens:499

Total Cost:0.006

In [45]:
response

In [51]:
quiz=response.get("quiz")


In [52]:
quiz=json.loads(quiz)


In [53]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [54]:
quiz_table_data

[]

{
    "1": {
        "question": "Which of the following industries uses Generative AI for creating new clothing designs and patterns?",
        "options": {
            "a": "Healthcare",
            "b": "Finance",
            "c": "Fashion",
            "d": "Creative Arts"
        },
        "correct_answer": "c"
    }
}

{
    "2": {
        "question": "What is one of the primary challenges associated with the use of Generative AI models?",
        "options": {
            "a": "High computational costs",
            "b": "Creation of deepfakes",
            "c": "Limited application areas",
            "d": "Lack of data sources"
        },
        "correct_answer": "b"
    }
}


In [55]:
quiz=pd.DataFrame(quiz_table_data)


In [58]:
quiz.to_csv("machinelearning.csv",index=False)